In [1]:
import pandas as pd
import numpy as np
import optuna
import dacon_law_class as dlc
from dacon_law_class import SimpleOps as so

import lightgbm as lgb
import re
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from xgboost.sklearn import XGBClassifier as xgb
# import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')


from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

 ___________________________
|                           |
|======== YearDream ========|
|===========================|
|==== DLC Well Imported ====|
|===========================|
|========= BYJASON =========|
|___________________________|



In [3]:
train = pd.read_csv('../train.csv')
test = pd.read_csv('../test.csv')
sample_submission = pd.read_csv('../sample_submission.csv')
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [4]:
# train
# test
sample_submission

,ID,first_party_winner
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
1235,TEST_1235,0
1236,TEST_1236,0
1237,TEST_1237,0
1238,TEST_1238,0


In [5]:
train.info()
print('\n\n\n')
# test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ID                  2478 non-null   object
 1   first_party         2478 non-null   object
 2   second_party        2478 non-null   object
 3   facts               2478 non-null   object
 4   first_party_winner  2478 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 96.9+ KB






In [6]:
df = pd.DataFrame(train['facts'])
df = dlc.law_preprocessor(df, 'facts')
train['facts'] = df['facts']
df = pd.DataFrame(train['first_party'])
df = dlc.law_preprocessor(df, 'first_party')
train['first_party'] = df['first_party']
df = pd.DataFrame(train['second_party'])
df = dlc.law_preprocessor(df, 'second_party')
train['second_party'] = df['second_party']
train_cleansed = train.drop(columns='ID')
train_cleansed


,first_party,second_party,facts,first_party_winner
0,phil amant,herman ompson,phil amant caidate for public office made tele...,1
1,ephen duncan,lawrence owens,ramon nelson was ridin his bike when he suffer...,0
2,billy joe mawood,tony patterson waen et al,an alabama ate court convicted billy joe mawoo...,1
3,linkletter,walker,victor linkletter was convicted in ate court o...,0
4,william earl fikes,alabama,in selma alabama an intruder broke into apartm...,1
...,...,...,...,...
2473,hollyfrontier cheyenne refinin llc et al,renewable fuels association et al,conress ameed clean air act rouh enery policy ...,1
2474,rupo mexicano de desarrollo,alliance bo fu inc,alliance bo fu inc an invement fu purchased ap...,1
2475,peuero,united ates,in dirict court sentenced manuel peuero to mon...,0
2476,iiration naturalization service,cyr,enrico cyr lawful permanent resident pled uilt...,0


In [7]:
df = pd.DataFrame(test['facts'])
df = dlc.law_preprocessor(df, 'facts')
test['facts'] = df['facts']
df = pd.DataFrame(test['first_party'])
df = dlc.law_preprocessor(df, 'first_party')
test['first_party'] = df['first_party']
df = pd.DataFrame(test['second_party'])
df = dlc.law_preprocessor(df, 'second_party')
test['second_party'] = df['second_party']
test_cleansed = test.drop(columns='ID')
test_cleansed

,first_party,second_party,facts
0,salerno,united ates,bail reform act allowed federal courts to deta...
1,milber weiss bershad hynes lerach,lexecon inc,lexecon inc was defeant in class action lawsui...
2,no title federal counications coission et al,fox television ations inc et al,in fox television ations broadca billboa music...
3,harold kaufman,united ates,durin his trial for armed robbery of federally...
4,berer,hanlon,in mairate jude issued warrant auorizin search...
...,...,...,...
1235,haitian centers council inc et al,chris sale actin coissioner iiration naturaliz...,accoin to executive oer no sined by president ...
1236,whitman,american truckin associations inc,section of clean air act requires environmenta...
1237,lia matteo john madian,william barr,lia matteo john madian created plan for utiliz...
1238,washinton ate apple advertisin coission,hunt,in nor carolina boa of ariculture adopted reul...


## BERT

@article{turc2019,
  title={Well-Read Students Learn Better: On the Importance of Pre-training Compact Models},
  author={Turc, Iulia and Chang, Ming-Wei and Lee, Kenton and Toutanova, Kristina},
  journal={arXiv preprint arXiv:1908.08962v2 },
  year={2019}
}


In [ ]:
train_facts = pd.DataFrame(test_cleansed['facts'])
train_facts

In [ ]:
test_fact = pd.DataFrame(test_cleansed['facts'])
test_fact

model_input_df = dlc.bert_tokenizer(train_facts, 'facts')

model_input_df.info()

df_part_1, df_part_2, df_part_3, df_part_4, df_part_5, df_part_6, df_part_7, df_part_8, df_part_9, df_part_10, df_part_11, df_part_12, df_part_13, df_part_14, df_part_15, df_part_16, df_part_17, df_part_18, df_part_19, df_part_20, df_part_21, df_part_22, df_part_23, df_part_24, df_part_25, df_part_26 = so.df_divider(train_cleansed, 'facts')
df_part_1 = pd.DataFrame(df_part_1)
df_part_2 = pd.DataFrame(df_part_2)
df_part_3 = pd.DataFrame(df_part_3)
df_part_4 = pd.DataFrame(df_part_4)
df_part_5 = pd.DataFrame(df_part_5)
df_part_6 = pd.DataFrame(df_part_6)
df_part_7 = pd.DataFrame(df_part_7)
df_part_8 = pd.DataFrame(df_part_8)
df_part_9 = pd.DataFrame(df_part_9)
df_part_10 = pd.DataFrame(df_part_10)
df_part_11 = pd.DataFrame(df_part_11)
df_part_12 = pd.DataFrame(df_part_12)
df_part_13 = pd.DataFrame(df_part_13)
df_part_14 = pd.DataFrame(df_part_14)
df_part_15 = pd.DataFrame(df_part_15)
df_part_16 = pd.DataFrame(df_part_16)
df_part_17 = pd.DataFrame(df_part_17)
df_part_18 = pd.DataFrame(df_part_18)
df_part_19 = pd.DataFrame(df_part_19)
df_part_20 = pd.DataFrame(df_part_20)
df_part_21 = pd.DataFrame(df_part_21)
df_part_22 = pd.DataFrame(df_part_22)
df_part_23 = pd.DataFrame(df_part_23)
df_part_24 = pd.DataFrame(df_part_24)
df_part_25 = pd.DataFrame(df_part_25)
df_part_26 = pd.DataFrame(df_part_26)
print(df_part_1)

embedded_df_1 = dlc.auto_tokenizer(train_cleansed, 'facts')
embedded_df_1

embedded_df_1 = embedded_df_1.rename(columns={0:'facts_berted'})

embedded_df_1.to_csv('./embeddings/facts_embedded.csv', index=False)

In [ ]:
first_party_berted = dlc.auto_tokenizer(train_cleansed, 'first_party')
first_party_berted = first_party_berted.rename(columns={0:'first_party_berted'})
first_party_berted.to_csv('./embeddings/first_party_berted.csv', index=False)

In [ ]:
second_party_berted = dlc.auto_tokenizer(train_cleansed, 'second_party')
second_party_berted = second_party_berted.rename(columns={0:'second_party_berted'})
second_party_berted.to_csv('./embeddings/second_party_berted.csv', index=False)

In [ ]:
facts_berted = dlc.auto_tokenizer(train_cleansed, 'facts')

In [ ]:
facts_berted = facts_berted.rename(columns={0:'facts_berted'})
facts_berted.to_csv('./embeddings/facts_berted.csv', index=False)

In [ ]:
first_party_berted = pd.read_csv('./embeddings/first_party_berted.csv')
second_party_berted = pd.read_csv('./embeddings/second_party_berted.csv')
facts_berted = pd.read_csv('./embeddings/facts_berted.csv')

In [ ]:
all_ready_to_ml = pd.concat([first_party_berted['first_party_berted'], second_party_berted['second_party_berted'], facts_berted['facts_berted'], train_cleansed['first_party_winner']], axis=1)
all_ready_to_ml.to_csv('./embeddings/1_train_ready_to_ml.csv', index=False)


In [ ]:
test_first_party_berted = dlc.auto_tokenizer(test_cleansed, 'first_party')
test_first_party_berted = test_first_party_berted.rename(columns={0:'first_party_berted'})
test_first_party_berted.to_csv('./embeddings/test_first_party_berted.csv', index=False)

In [ ]:
test_second_party_berted = dlc.auto_tokenizer(test_cleansed, 'second_party')
test_second_party_berted = test_second_party_berted.rename(columns={0:'second_party_berted'})
test_second_party_berted.to_csv('./embeddings/test_second_party_berted.csv', index=False)

In [ ]:
test_facts_berted = dlc.auto_tokenizer(test_cleansed, 'facts')
test_facts_berted = test_facts_berted.rename(columns={0:'facts_berted'})
test_facts_berted.to_csv('./embeddings/test_facts_berted.csv', index=False)

In [ ]:
test_first_party_berted = pd.read_csv('./embeddings/test_first_party_berted.csv')
test_second_party_berted = pd.read_csv('./embeddings/test_second_party_berted.csv')
test_facts_berted = pd.read_csv('./embeddings/test_facts_berted.csv')

In [ ]:
test_ready_to_ml = pd.concat([test_first_party_berted['first_party_berted'], test_second_party_berted['second_party_berted'], test_facts_berted['facts_berted']], axis=1)
test_ready_to_ml.to_csv('./embeddings/2_test_ready_to_ml.csv', index=False)

# 여기

In [10]:
train_to_ml = pd.read_csv('../embeddings/1_train_ready_to_ml.csv')
test_ready_to_ml = pd.read_csv('../embeddings/2_test_ready_to_ml.csv')

In [11]:
train_to_ml_fp_pr = dlc.tensor_separator(train_to_ml, 'first_party_berted')
train_to_ml_sp_pr = dlc.tensor_separator(train_to_ml, 'second_party_berted')
train_to_ml_facts_pr = dlc.tensor_separator(train_to_ml, 'facts_berted')

100%|██████████████████████| 2478/2478 [00:01<00:00, 2259.31it/s]


In [12]:
train_to_ml_fp_pr = train_to_ml_fp_pr.astype('float64')
train_to_ml_sp_pr = train_to_ml_sp_pr.astype('float64') 
train_to_ml_facts_pr = train_to_ml_facts_pr.astype('float64')


In [13]:
train_to_ml_fp_pr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Columns: 768 entries, 0 to 767
dtypes: float64(768)
memory usage: 14.5 MB


In [14]:
test_to_ml_fp_pr = dlc.tensor_separator(test_ready_to_ml, 'first_party_berted')
test_to_ml_sp_pr = dlc.tensor_separator(test_ready_to_ml, 'second_party_berted')
test_to_ml_facts_pr = dlc.tensor_separator(test_ready_to_ml, 'facts_berted')

100%|██████████████████████| 1240/1240 [00:00<00:00, 2254.71it/s]


In [15]:
test_to_ml_fp_pr = test_to_ml_fp_pr.astype('float64')
test_to_ml_sp_pr = test_to_ml_sp_pr.astype('float64')
test_to_ml_facts_pr = test_to_ml_facts_pr.astype('float64')

In [16]:
to_be_X = pd.concat([train_to_ml_fp_pr, train_to_ml_facts_pr], axis=1)
to_be_test_x = pd.concat([test_to_ml_fp_pr, test_to_ml_facts_pr], axis=1)


In [17]:
to_be_X.columns = np.arange(len(to_be_X.columns))
to_be_X = pd.concat([to_be_X, train_to_ml['first_party_winner']], axis =1)
to_be_X

,0,1,2,3,4,5,6,7,8,9,...,1527,1528,1529,1530,1531,1532,1533,1534,1535,first_party_winner
0,0.005036,0.006439,-0.012888,-0.007303,-0.017571,-0.029552,-0.006318,0.025047,-0.009394,-0.015124,...,-0.018174,0.001737,0.003518,-0.010968,-0.030724,-0.035971,-0.038689,-0.006275,-0.010507,1
1,-0.003902,0.026781,-0.044628,-0.032344,-0.001729,-0.018735,0.002111,-0.018207,-0.009919,-0.075807,...,-0.010969,0.006232,-0.012417,-0.020754,-0.016356,-0.039208,-0.031188,-0.014293,-0.026212,0
2,0.003050,0.009259,-0.017064,-0.019458,-0.006200,-0.008553,0.050010,0.035903,-0.030393,-0.020106,...,-0.011412,-0.006155,-0.010546,-0.024606,-0.025278,-0.054787,-0.016296,-0.009530,-0.012053,1
3,-0.015234,-0.000875,0.015897,-0.018534,0.054608,-0.030493,0.041842,0.057857,-0.013273,-0.019490,...,-0.019637,-0.005936,-0.003335,-0.026111,-0.015403,0.000960,-0.027738,0.005133,-0.012935,0
4,-0.011956,-0.009987,0.020198,0.009508,0.024592,-0.037784,0.020635,-0.023835,-0.029138,-0.031670,...,-0.033741,0.007560,-0.023639,-0.028438,-0.010833,-0.030627,-0.035033,-0.000732,-0.024343,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,-0.020591,-0.056972,-0.033879,0.031382,0.079985,0.025763,-0.006569,0.044251,-0.008797,-0.060979,...,-0.027422,0.000112,0.015122,0.009355,-0.046905,-0.002683,-0.008214,-0.000282,-0.009470,1
2474,-0.023678,-0.022937,-0.022214,0.013166,0.054840,0.011287,0.016339,0.090016,0.000314,0.001474,...,-0.017667,0.015822,-0.002527,0.004892,-0.039568,-0.052614,-0.034712,-0.004466,0.008349,1
2475,-0.004295,-0.029256,-0.015316,-0.010641,0.007141,0.021234,0.005180,0.064935,-0.022453,-0.034121,...,-0.011819,0.019265,-0.019696,-0.024496,-0.042203,-0.044682,-0.023756,-0.004682,0.003538,0
2476,0.021019,-0.003120,-0.007031,0.007962,0.045658,0.014659,0.024269,-0.021894,-0.032928,-0.029230,...,-0.023799,0.002132,-0.011824,-0.020932,-0.033988,-0.028924,-0.045331,-0.013245,-0.016124,0


In [18]:
to_be_test_x.columns = np.arange(len(to_be_test_x.columns))
to_be_test_x

,0,1,2,3,4,5,6,7,8,9,...,1526,1527,1528,1529,1530,1531,1532,1533,1534,1535
0,-0.006672,-0.080011,-0.014580,-0.038840,0.048455,-0.040639,0.039663,0.068547,0.002239,-0.029425,...,-0.012600,-0.023597,-0.003138,-0.008353,-0.034796,-0.020804,-0.006913,-0.038063,0.008657,-0.021999
1,-0.003053,-0.011769,-0.034803,-0.041324,-0.026498,-0.014561,0.012917,-0.009658,-0.000563,-0.018914,...,-0.018383,-0.027521,0.004810,-0.006279,-0.019910,-0.044318,-0.049594,-0.022413,0.012250,0.003738
2,-0.025638,-0.016005,-0.021478,0.014413,0.048054,-0.036683,-0.034054,0.043429,-0.017806,-0.045973,...,-0.023804,-0.017397,0.008190,-0.010989,-0.014011,-0.041264,-0.009615,-0.012293,-0.000732,0.010219
3,0.028991,0.036599,-0.059128,-0.030722,-0.017587,-0.028222,0.052064,-0.009327,0.006705,-0.008745,...,-0.021711,-0.015069,-0.001111,-0.014473,-0.025876,-0.006428,-0.018803,-0.028281,-0.004001,-0.012030
4,0.036941,-0.039247,-0.026675,-0.006587,-0.022472,-0.016113,0.046348,-0.012971,0.029547,-0.051456,...,-0.016862,-0.034918,0.005033,0.000040,-0.037341,-0.040531,-0.014745,-0.042890,-0.000467,-0.010147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,-0.017606,-0.026507,-0.038503,-0.013017,0.036128,-0.006750,-0.023582,0.004234,-0.010011,-0.024067,...,-0.010572,-0.034447,-0.002972,-0.012302,-0.004949,-0.046846,-0.039191,-0.034175,-0.013973,0.006327
1236,-0.001220,-0.002364,-0.025906,-0.023937,0.002847,-0.021732,0.076602,-0.014772,-0.012813,-0.014318,...,-0.000953,-0.013990,0.002872,-0.030309,-0.013520,-0.053392,-0.012986,-0.014506,-0.011636,0.024630
1237,0.001026,0.023440,0.013377,0.016586,0.024960,-0.001395,0.063505,-0.017101,-0.032935,-0.016052,...,-0.018592,-0.016234,0.018842,-0.002355,-0.015415,-0.033917,-0.057270,-0.028927,-0.005387,0.000249
1238,0.001634,-0.000814,-0.014695,0.003300,0.001420,0.028769,-0.025222,0.065449,-0.011581,-0.044858,...,0.004752,0.003640,0.013376,-0.006307,-0.005089,-0.044533,-0.016824,-0.018459,-0.004400,0.022938


In [19]:
X = to_be_X.drop(columns='first_party_winner')
y = pd.DataFrame(to_be_X['first_party_winner'])

In [20]:
test_x = to_be_test_x

In [21]:
X.info()

print('\n --------------------------------------- \n')

y.info()

print('\n --------------------------------------- \n')

test_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Columns: 1536 entries, 0 to 1535
dtypes: float64(1536)
memory usage: 29.0 MB

 --------------------------------------- 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2478 entries, 0 to 2477
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   first_party_winner  2478 non-null   int64
dtypes: int64(1)
memory usage: 19.5 KB

 --------------------------------------- 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1240 entries, 0 to 1239
Columns: 1536 entries, 0 to 1535
dtypes: float64(1536)
memory usage: 14.5 MB


In [25]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

(1734, 1536) (744, 1536) (1734, 1) (744, 1)


In [27]:
def xgb_objective(trial):
    xgb_params = {
        'objective': 'binary:logistic',
        'eval_metric': 'error',
        'booster': 'gbtree',
        'nthread': trial.suggest_int('nthread', 1, 15),
        'n_estimators' : trial.suggest_int('n_estimators', 25, 1000),
        'max_depth': trial.suggest_int('max_depth', 4, 15),
        'subsample': trial.suggest_uniform('subsample', 0.1, 0.3),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.5),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 0.7),
        'lambda': trial.suggest_loguniform('lambda', 0.2, 200),
        'random_state': 42,
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 3)
    }
    


    xgb_model = xgb(**xgb_params)
    xgb_model.fit(X_train, y_train)
    xgb_preds = xgb_model.predict(X_val)
    
    return accuracy_score(y_val, xgb_preds)


In [26]:
def lgb_objective(trial):
    lgb_params = {
        'application': 'binary',
        'max_depth': -1,
        'metric': 'binary_logloss',
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt',  'dart']),
        'num_leaves': trial.suggest_int('num_leaves', 10, 2000),
        'lambda' : trial.suggest_float('lambda', 0.01, 0.5),
        'num_iteration': 500,
        'n_jobs': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.05, 0.1),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.7, 0.9),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 0.8),
        'random_state': 42
    }
    
    lgb_model = lgb.LGBMClassifier(**lgb_params)
    lgb_model.fit(X_train, y_train)
    lgb_preds = lgb_model.predict(X_val)
    
    return accuracy_score(y_val, lgb_preds)


In [29]:
def cat_objective(trial):
    params = {
            'loss_function': 'Logloss',
            'learning_rate': learning_rate,
            'depth': trial.suggest_int('depth', 3, 10),
            'random_state': 42
        }


    model = cat.CatBoostClassifier(**params)
    model.fit(X_train, y_train)
    pred = model.predict(X_val)
    
    return accuracy_score(y_val, pred)

In [ ]:
optimizes = dlc.optimize

In [25]:
study = optuna.create_study(direction='minimize')

[I 2023-06-10 15:52:33,476] A new study created in memory with name: no-name-5499f085-f560-452d-b436-ad41805ba3f3


In [27]:
lgb_study = optuna.create_study(direction='minimize')
lgb_study.optimize(lgb_objective, n_trials=5, show_progress_bar=True)

[I 2023-06-11 13:37:13,432] A new study created in memory with name: no-name-3f64f5dd-cf6d-41d0-a6f5-b7de1f5fe317


  0%|          | 0/5 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.07911695061300714. Current value: lambda_l2=0.07911695061300714
[LightGBM] [Warning] num_iterations is set=500, num_iteration=500 will be ignored. Current value: num_iterations=500
[LightGBM] [Warning] feature_fraction is set=0.8318072994354647, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8318072994354647
[LightGBM] [Warning] bagging_fraction is set=0.6772159567546318, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6772159567546318
[I 2023-06-11 13:37:38,521] Trial 0 finished with value: 0.6599462365591398 and parameters: {'boosting_type': 'dart', 'num_leaves': 1921, 'lambda': 0.07911695061300714, 'learning_rate': 0.07521150631221399, 'feature_fraction': 0.8318072994354647, 'bagging_fraction': 0.6772159567546318}. Best is trial 0 with value: 0.6599462365591398.
[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.42

print('Number of finished XGB trials: {}'.format(len(xgb_study.trials)))
print('XGB Best trial:')
xgb_trial = xgb_study.best_trial

print('  Value: {}'.format(xgb_trial.value))
print('  Params: ')

for key, value in xgb_trial.params.items():
    print('    {}: {}'.format(key, value))

In [28]:
print('Number of finished LGB trials: {}'.format(len(lgb_study.trials)))
print('LGB Best trial:')
lgb_trial = lgb_study.best_trial

print('  Value: {}'.format(lgb_trial.value))
print('  Params: ')

for key, value in lgb_trial.params.items():
    print('    {}: {}'.format(key, value))


Number of finished LGB trials: 5
LGB Best trial:
  Value: 0.6559139784946236
  Params: 
    boosting_type: gbdt
    num_leaves: 1611
    lambda: 0.29749127298451106
    learning_rate: 0.06593896755655602
    feature_fraction: 0.7542207041219193
    bagging_fraction: 0.7099047625020872


In [29]:
lgb_best_params = lgb_study.best_params
lgb_best_params['random_state'] = 42
lgb_model = lgb.LGBMClassifier(**lgb_best_params)
lgb_model.fit(X_train, y_train)

[LightGBM] [Warning] lambda_l2 is set with reg_lambda=0.0, will be overridden by lambda=0.29749127298451106. Current value: lambda_l2=0.29749127298451106
[LightGBM] [Warning] feature_fraction is set=0.7542207041219193, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7542207041219193
[LightGBM] [Warning] bagging_fraction is set=0.7099047625020872, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7099047625020872


LGBMClassifier(bagging_fraction=0.7099047625020872,
               feature_fraction=0.7542207041219193, lambda=0.29749127298451106,
               learning_rate=0.06593896755655602, num_leaves=1611,
               random_state=42)

In [30]:
lgb_preds = lgb_model.predict(X_val)
lgb_accuracy = accuracy_score(y_val, lgb_preds)

XGB_pred = XGB.predict(X_val)
accuracy = accuracy_score(y_val, XGB_pred)
print("\nAccuracy after tuning: %.2f%%" % (accuracy * 100.0))

In [32]:
print("-- LGB --")
print("Train ACC : %.3f" % accuracy_score(y_train, lgb_model.predict(X_train)))
print("Val ACC : %.3f" % accuracy_score(y_val, lgb_model.predict(X_val)))

-- LGB --
Train ACC : 1.000
Val ACC : 0.656


print("\n-- XGB --")
print("Train ACC : %.3f" % accuracy_score(y_train, XGB.predict(X_train)))
print("Val ACC : %.3f" % accuracy_score(y_val, XGB.predict(X_val)))

In [35]:
print(classification_report(y_val, lgb_preds))


              precision    recall  f1-score   support

           0       0.38      0.07      0.12       245
           1       0.67      0.94      0.79       499

    accuracy                           0.66       744
   macro avg       0.53      0.51      0.45       744
weighted avg       0.58      0.66      0.57       744



In [37]:
X_test = pd.get_dummies(data=test_x)
print(X_test)
lgb_preds = lgb_model.predict(test_x)

          0         1         2         3         4         5         6     \
0    -0.006672 -0.080011 -0.014580 -0.038840  0.048455 -0.040639  0.039663   
1    -0.003053 -0.011769 -0.034803 -0.041324 -0.026498 -0.014561  0.012917   
2    -0.025638 -0.016005 -0.021478  0.014413  0.048054 -0.036683 -0.034054   
3     0.028991  0.036599 -0.059128 -0.030722 -0.017587 -0.028222  0.052064   
4     0.036941 -0.039247 -0.026675 -0.006587 -0.022472 -0.016113  0.046348   
...        ...       ...       ...       ...       ...       ...       ...   
1235 -0.017606 -0.026507 -0.038503 -0.013017  0.036128 -0.006750 -0.023582   
1236 -0.001220 -0.002364 -0.025906 -0.023937  0.002847 -0.021732  0.076602   
1237  0.001026  0.023440  0.013377  0.016586  0.024960 -0.001395  0.063505   
1238  0.001634 -0.000814 -0.014695  0.003300  0.001420  0.028769 -0.025222   
1239 -0.058938  0.010668  0.004917 -0.051615  0.033669  0.037903  0.019593   

          7         8         9     ...      1526      1527    

In [39]:
sample_submission = pd.read_csv('../sample_submission.csv')
LGB_pred = pd.DataFrame(lgb_preds.astype(int))
LGB_pred = LGB_pred.rename(columns={0:'first_party_winner'})

In [40]:
LGB_pred

,first_party_winner
0,1
1,1
2,1
3,1
4,1
...,...
739,1
740,1
741,1
742,1


In [41]:
sample_submission['first_party_winner'] = LGB_pred['first_party_winner']

In [43]:
print(sample_submission['first_party_winner'].value_counts())

1.0    697
0.0     47
Name: first_party_winner, dtype: int64


In [42]:
sample_submission.to_csv("./results/LGB_submission_{Train:.03f}_{Val:.03f}.csv".format(Train=accuracy_score(y_train, lgb_model.predict(X_train)), Val = accuracy_score(y_val, lgb_model.predict(X_val))), index=False)

In [ ]:
# XGB_submission = pd.read_csv('./sample_submission.csv')
# XGB_pred = pd.DataFrame(XGB_pred.astype(int))
# XGB_submission['first_party_winner'] = XGB_pred
# XGB_submission.to_csv("./Bert_XGB_submission_{Train:.03f}_{Val:.03f}.csv".format(Train=accuracy_score(y_train, XGB.predict(X_train)), Val = accuracy_score(y_val, XGB.predict(X_val))), index=False)

In [ ]:
XGB_submission

In [ ]:
LGB_submission

In [ ]:
# 3D to 2D

attention_mask_df = dlc.tensor_2_2d(train_bert_tokenized, 0)
input_ids_df = dlc.tensor_2_2d(train_bert_tokenized, 1)
token_type_ids_df = dlc.tensor_2_2d(train_bert_tokenized, 2)

attention_mask_df.info()
print('\n _______________________________ \n')
input_ids_df.info()
print('\n _______________________________ \n')
token_type_ids_df.info()


In [ ]:
# attention_mask_df.info()
attention_mask_df
# input_ids_df.info()
# input_ids_df
# token_type_ids_df.info()
# token_type_ids_df


In [ ]:
temp = pd.DataFrame()
temp = pd.concat([train_cleansed['ID'], attention_mask_df], axis=1)
temp = pd.concat([temp, input_ids_df], axis=1)
train_BertToken_df = pd.concat([temp, token_type_ids_df], axis=1)
train_BertToken_df


In [ ]:
tBTdf = so.right_merger(train_cleansed, train_BertToken_df, 0)
tBTdf